In [12]:
import re
import json
import os
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='ko')

def translate_text(text):
    if not text or not isinstance(text, str):
        return text

    # 번역 제외 대상: @UUID[...] 와 &amp;Reference[...]
    chunks = re.split(r'(&amp;Reference\[.*?\]|@UUID\[.*?\]{.*?})', text)
    translated_chunks = []

    for chunk in chunks:
        if re.match(r'(&amp;Reference\[.*?\]|@UUID\[.*?\]{.*?})', chunk):
            translated_chunks.append(chunk)  # 번역 안함
        else:
            # HTML 태그 보존
            parts = re.split(r'(<[^>]+>)', chunk)
            for part in parts:
                if re.match(r'<[^>]+>', part) or not part.strip():
                    translated_chunks.append(part)
                else:
                    try:
                        translated = translator.translate(part)
                        translated_chunks.append(translated if translated else part)
                    except Exception:
                        translated_chunks.append(part)

    return ''.join(translated_chunks)

def translate_name_or_title(value):
    try:
        translated = translator.translate(value)
        return f"{translated}({value})" if translated else value
    except Exception:
        return value

def recursive_translate(data):
    if isinstance(data, dict):
        translated = {}
        for k, v in data.items():
            if k in ['name', 'title']:
                translated[k] = translate_name_or_title(v)
            elif k in ['description', 'hint', 'condition']:
                translated[k] = translate_text(v)
            else:
                translated[k] = recursive_translate(v)
        return translated
    elif isinstance(data, list):
        return [recursive_translate(item) for item in data]
    elif isinstance(data, str):
        return data
    else:
        return data

# 파일 경로
input_path = "./dnd-players-handbook.classes.json"
output_path = "./dnd-players-handbook.classes.translated.json"

# JSON 로드
with open(input_path, "r", encoding="utf-8") as f:
    source_json = json.load(f)

# 번역 수행
translated_json = recursive_translate(source_json)

# 디렉터리 없으면 생성 후 저장
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_json, f, ensure_ascii=False, indent=2)

print("✅ 번역 및 저장 완료!")


✅ 번역 및 저장 완료!
